#### Exercício 1: Carregando e analisando um DataSet


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

'''
Carregue os dados das três estações em DataFrame s separados e analise a
estrutura dos mesmos utilizando o método head() 
'''
# Carrega datasets
arquivo1 = "BAURU_01-01-2003_A_31-12-2003.csv"
arquivo2 = "BAURU_01-01-2013_A_31-12-2013.csv"
arquivo3 = "BAURU_01-01-2023_A_31-12-2023.csv"

df1 = pd.read_csv(arquivo1, delimiter=";", decimal=",")
df2 = pd.read_csv(arquivo2, delimiter=";", decimal=",")
df3 = pd.read_csv(arquivo3, delimiter=";", decimal=",")

df1.replace(-9999, 0, inplace=True)
df2.replace(-9999, 0, inplace=True)
df3.replace(-9999, 0, inplace=True)

print("DataFrame 2003:")
print(df1.head())
print("\nDataFrame 2013:")
print(df2.head())
print("\nDataFrame 2023:")
print(df3.head())


DataFrame 2003:
         Data Hora UTC  PRECIPITACAO TOTAL, HORARIO (mm)  \
0  2003-01-01    00:00                               0.0   
1  2003-01-01    01:00                               0.0   
2  2003-01-01    02:00                               0.0   
3  2003-01-01    03:00                               0.0   
4  2003-01-01    04:00                               0.0   

   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  \
0                                              940.2       
1                                              941.1       
2                                              940.9       
3                                              940.5       
4                                              940.0       

   PRESSAO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)  \
0                                            940.3   
1                                            941.1   
2                                            941.1   
3                                            9

#### Exercício 2: Juntando os datasets

In [2]:
'''
Construa um novo DataFrame juntando os mesmos e utilizando um MultiIndex como índice 
formado pelos os anos, os messes de cada ano e os dias de cada mes.
'''
def tratamento_hora (string):
    string = string.replace("/","-")
    return string

df3["Data"] = df3["Data"].astype("string")
df3["Data"] = df3["Data"].apply(tratamento_hora)
concatdf = pd.concat([df1, df2, df3], axis=0)
concatdf["Data"] = pd.to_datetime(concatdf["Data"])
concatdf = concatdf.reset_index()

ano = concatdf['Data'].dt.year
mes = concatdf['Data'].dt.month
dia = concatdf['Data'].dt.day

concatdf = concatdf.set_index(pd.MultiIndex.from_arrays([dia, mes, ano], names=('dia', 'mes', 'ano')))
concatdf.drop('Data', axis=1, inplace=True)

'''
Construa um novo DataFrame modificando o anterior para utiliza um
MultiIndex como colunas contendo os parâmetros agrupados (precipitações,
pressão, radiação, temperatura, umidade, velocidade do vento )
'''

arrayc = []
for columns in concatdf.columns:
    a = (columns,"")
    if(columns.find("PRESSAO")>-1 or columns.find("PRESSÃO")>-1):
        a = ("Pressão",columns)
    if(columns.find("TEMPERATURA")>-1):
        a = ("Temperatura",columns)
    if(columns.find("UMIDADE")>-1):
        a = ("Umidade",columns)
    if(columns.find("VENTO")>-1):
        a = ("Velocidade do Vento",columns)
    arrayc.append(a)
    
concatdf.columns = pd.MultiIndex.from_tuples(arrayc)
concatdf

index  Hora UTC PRECIPITACAO TOTAL, HORARIO (mm)  \
                                                                
dia mes ano                                                     
1   1   2003     0     00:00                              0.0   
        2003     1     01:00                              0.0   
        2003     2     02:00                              0.0   
        2003     3     03:00                              0.0   
        2003     4     04:00                              0.0   
...            ...       ...                              ...   
31  12  2023  8755  1900 UTC                              0.0   
        2023  8756  2000 UTC                              0.0   
        2023  8757  2100 UTC                              0.0   
        2023  8758  2200 UTC                              0.0   
        2023  8759  2300 UTC                              0.0   

                                                           Pressão  \
             PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)   
dia mes ano                                                          
1   1   2003                                              940.2      
        2003                                              941.1      
        2003                                              940.9      
        2003                                              940.5      
        2003                                              940.0      
...                                                         ...      
31  12  2023                                              939.6      
        2023                                              939.5      
        2023                                              939.4      
        2023                                              939.6      
        2023                                              940.5      

                                                              \
             PRESSAO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)   
dia mes ano                                                    
1   1   2003                                           940.3   
        2003                                           941.1   
        2003                                           941.1   
        2003                                           941.0   
        2003                                           940.5   
...                                                      ...   
31  12  2023                                           940.6   
        2023                                           939.6   
        2023                                           939.6   
        2023                                           939.6   
        2023                                           940.5   

                                                               \
             PRESSAO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)   
dia mes ano                                                     
1   1   2003                                            939.5   
        2003                                            940.3   
        2003                                            940.8   
        2003                                            940.5   
        2003                                            940.0   
...                                                       ...   
31  12  2023                                            939.5   
        2023                                            939.2   
        2023                                            939.1   
        2023                                            939.3   
        2023                                            939.6   

             RADIACAO GLOBAL (Kj/m2)  \
                                       
dia mes ano                            
1   1   2003                     0.0   
        2003                     0.0   
        2003                     0.0   
        2003                     0.0   
        2003                     

#### Exercise 3: Analisando os DataFrame

In [3]:
'''
Analisando as colunas relacionadas e precipitações e temperaturas, compare as
médias, os valores mínimos e máximos em cada um dos anos.
'''

# Análise de precipitação
print(concatdf.groupby('ano')['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))
# Análise de temperatura
print(concatdf.groupby('ano')['Temperatura'].agg(['mean', 'min', 'max']))

          mean  min   max
ano                      
2003  0.142945  0.0  37.6
2013  0.152397  0.0  37.6
2023  0.127983  0.0  30.2
                                     Temperatura             \
     TEMPERATURA DO AR - BULBO SECO, HORARIA (C)              
                                            mean  min   max   
ano                                                           
2003                                   19.072466  0.0  36.8   
2013                                   17.337511  0.0  35.1   
2023                                   22.415184  6.3  39.1   

                                                     \
     TEMPERATURA DO PONTO DE ORVALHO (C)              
                                    mean  min   max   
ano                                                   
2003                           12.986119 -1.3  24.3   
2013                           11.267637 -2.5  25.6   
2023                           16.328782  4.3  27.6   

                                          

In [9]:
'''
Compare os mesmos dados do exercício anterior mas para o mes de Janeiro de
cada um dos três anos.
'''
print(concatdf.xs(1, level ='mes' ).groupby('ano')['Temperatura'].agg(['mean', 'min', 'max']))
print(concatdf.xs(1, level ='mes' ).groupby('ano')['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))


                                     Temperatura              \
     TEMPERATURA DO AR - BULBO SECO, HORARIA (C)               
                                            mean   min   max   
ano                                                            
2003                                   18.075403   0.0  33.1   
2013                                   22.875403  15.4  32.2   
2023                                   23.228937  16.9  32.6   

                                                      \
     TEMPERATURA DO PONTO DE ORVALHO (C)               
                                    mean   min   max   
ano                                                    
2003                            15.07164   0.0  24.3   
2013                            14.74543   0.0  25.3   
2023                            19.72619  13.5  24.1   

                                                            \
     TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C)               
                                  

In [8]:
'''
Considere uma aproximação para as estações do ano no brasil dada por: Verão
(Janeiro, Fevereiro e Março), Outono (Abril, Maio e Junho), Inverno (Julho, Agosto e
Setembro) Primavera (Outubro, Novembro e Dezembro). Compare as informações
sobre temperatura e precipitações medias em cada uma das estações dos três
anos.
'''
concatdf['estacao'] = ["verao" if m in [1, 2, 3] else "outono" if m in [4, 5, 6]
                       else "inverno" if m in [7,8,9]
                       else "primavera" if m in [10,11,12] else""
                       for m in concatdf.index.get_level_values('mes')]
print("----------------2003---------------")
print(concatdf.xs(2003, level ='ano').groupby(['estacao'])['Temperatura'].agg(['mean', 'min', 'max']))
print(concatdf.xs(2003, level ='ano').groupby('estacao')['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))
print("----------------2013---------------")
print(concatdf.xs(2013, level ='ano').groupby('estacao')['Temperatura'].agg(['mean', 'min', 'max']))
print(concatdf.xs(2013, level ='ano').groupby('estacao')['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))
print("----------------2023---------------")
print(concatdf.xs(2023, level ='ano').groupby('estacao')['Temperatura'].agg(['mean', 'min', 'max']))
print(concatdf.xs(2023, level ='ano').groupby('estacao')['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))

----------------2003---------------
                                          Temperatura             \
          TEMPERATURA DO AR - BULBO SECO, HORARIA (C)              
                                                 mean  min   max   
estacao                                                            
inverno                                     19.149049  0.0  36.8   
outono                                      17.514011  0.0  31.8   
primavera                                   22.117029  0.0  35.8   
verao                                       17.457731  0.0  35.6   

                                                          \
          TEMPERATURA DO PONTO DE ORVALHO (C)              
                                         mean  min   max   
estacao                                                    
inverno                             10.273505 -1.3  18.2   
outono                              12.183333  0.0  20.8   
primavera                           15.453080  0.0  22.5   

In [7]:
'''
Repare que os dados de cada estação são monitorando de hora em hora. Considere
os intervalos de um dia como sendo: Madrugada (00:00 a 06:00), Manha (06:00 a
12:00), Tarde (12:00 a 18:00) e Noite (18:00 a 00:00). Compare as informações sobre
temperatura e precipitações medias para o dia 28 de setembro em cada um dos
intervalos do dia dos três anos.
'''
concatdf['Hora UTC'] =concatdf['Hora UTC'].astype('string')

def forcabruta(string):
    nova_string = string[:2]
    return nova_string
    
concatdf['Hora UTC'] = concatdf['Hora UTC'].apply(forcabruta)
concatdf['Hora UTC'] = concatdf['Hora UTC'].astype('int')

concatdf['periodo'] = ['madrugada' if (0 <= t and t < 6) else 'manha' if (6 <= t and t < 12)
                       else 'tarde' if (12 <= t and t < 18) else 'noite' if (18 <= t)
                       else '' for t in concatdf['Hora UTC']]

print("----------------2003---------------")
print(concatdf.xs((2003, 9, 28), level=('ano', 'mes', 'dia')).groupby(['periodo'])['Temperatura'].agg(['mean', 'min', 'max']))
print(concatdf.xs((2003, 9, 28), level=('ano', 'mes', 'dia')).groupby('periodo')['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))
print("----------------2013---------------")
print(concatdf.xs((2013, 9, 28), level=('ano', 'mes', 'dia')).groupby(['periodo'])['Temperatura'].agg(['mean', 'min', 'max']))
print(concatdf.xs((2013, 9, 28), level=('ano', 'mes', 'dia')).groupby(['periodo'])['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))
print("----------------2023---------------")
print(concatdf.xs((2023, 9, 28), level=('ano', 'mes', 'dia')).groupby(['periodo'])['Temperatura'].agg(['mean', 'min', 'max']))
print(concatdf.xs((2023, 9, 28), level=('ano', 'mes', 'dia')).groupby(['periodo'])['PRECIPITACAO TOTAL, HORARIO (mm)'].agg(['mean', 'min', 'max']))


----------------2003---------------
                                          Temperatura              \
          TEMPERATURA DO AR - BULBO SECO, HORARIA (C)               
                                                 mean   min   max   
periodo                                                             
madrugada                                   20.366667  19.7  21.1   
manha                                       18.116667  17.2  20.7   
noite                                       28.666667  24.1  32.7   
tarde                                       28.650000  24.1  32.1   

                                                           \
          TEMPERATURA DO PONTO DE ORVALHO (C)               
                                         mean   min   max   
periodo                                                     
madrugada                           14.700000  14.1  15.1   
manha                               14.633333  14.4  15.4   
noite                                9.100000